# Specifying CURL command and creating a python script of it

In [143]:
import json

def get_article_list(cmcontinue=''):
    headers = {
        'Accept-Encoding': 'gzip,deflate',
    }

    params = (
        ('action', 'query'),
    )

    data = {
      'list': 'categorymembers',
      'cmtitle': 'Category:Featured_articles',
      'cmlimit': '500',
      'action': 'query',
      'format' : 'json',
      'cmcontinue' : cmcontinue
    }

    response = requests.post('https://en.wikipedia.org/w/api.php', headers=headers,data=data)
    d = json.loads(response.content)
    if 'continue' in d.keys():
        cmcontinue = d['continue']['cmcontinue']
    else:
        cmcontinue = ''
    titles = [elem['title'] for elem in d['query']['categorymembers']]
    return titles, cmcontinue

titles, cmcontinue = get_article_list()
featured_articles = titles

while len(titles) == 500:
    titles, cmcontinue = get_article_list(cmcontinue)
    featured_articles.extend(titles)
    print(len(featured_articles))

for idx, title in enumerate(featured_articles):
    featured_articles[idx] = title.replace(' ','_')


1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6141


In [144]:
import requests
from io import BytesIO


def retrieve_article_revisions(articlename, timestamp=''):
    
    headers = {
        'Accept-Encoding': 'gzip,deflate',
    }

    params = (
        ('title', 'Special:Export'),
    )

    data = {
      'pages': articlename,
      'offset': timestamp,
      'limit': '1000',
      'action': 'submit'
    }

    response = requests.post('https://en.wikipedia.org/w/index.php', headers=headers, params=params, data=data)
    return response.content


def retrieve_article(articlename):
    timestamp = ''
    xml_list = []
    num = 1000
    
    while num == 1000:
        data = retrieve_article_revisions(articlename, timestamp)
        timestamp, num = get_timestamp_from_xml(BytesIO(data))
        print('Retrieved {} revisions from article about {}, last from {}'.format(num, articlename, timestamp))
        xml_list.append(data)
    
    return xml_list

In [145]:
featured_articles_revisions = {}

for article in featured_articles[40:50]:
    featured_articles_revisions[article] = retrieve_article(article)

Retrieved 868 revisions from article about 243_Ida, last from 2022-09-11T05:51:35Z
Retrieved 687 revisions from article about 509th_Composite_Group, last from 2022-07-19T23:48:08Z
Retrieved 231 revisions from article about 766th_Independent_Infantry_Regiment_(North_Korea), last from 2022-08-06T17:52:42Z
Retrieved 755 revisions from article about 1080°_Snowboarding, last from 2022-09-13T15:25:45Z
Retrieved 212 revisions from article about 1678_Kediri_campaign, last from 2022-08-06T17:55:46Z
Retrieved 624 revisions from article about 1740_Batavia_massacre, last from 2022-09-04T04:54:31Z
Retrieved 410 revisions from article about 1789_Virginia's_5th_congressional_district_election, last from 2022-06-14T20:27:51Z
Retrieved 653 revisions from article about 1804_dollar, last from 2022-06-14T20:28:01Z
Retrieved 673 revisions from article about 1838_Jesuit_slave_sale, last from 2022-07-14T23:19:12Z
Retrieved 123 revisions from article about 1850_Atlantic_hurricane_season, last from 2022-06-14T

# Writing the xml files in local database

In [146]:
for key in featured_articles_revisions.keys():
    for idx,revision in enumerate(featured_articles_revisions[key]):
        filename = "{}_{}.xml".format(key,idx)
        with open(filename, 'wb') as f:
            f.write(revision)
            